<a href="https://colab.research.google.com/github/PixelMD97/RAGforICUResearch/blob/main/Weiterbildung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [68]:
import pandas as pd
import re
from datetime import date

# PEP

In [69]:
ROLE_MAP = {
    "Chefarzt/ärztin Unispital": "CA",
    "Stv.Chefarzt/ärztin Unispital": "SCA",
    "Lt.e/r Arzt/Ärztin Unispital": "LA",
    "Spit.facharzt/tin I Unispital": "SFA_I",
    "Spit.facharzt/tin II Unispit.": "SFA_II",
    "Oberarzt/ärztin I Unispital": "OA_I",
    "Oberarzt/ärztin II Unispital": "OA_II",
    "Stv. Oberarzt/ärztin Unispit.": "SOA",
    "Assistenzarzt/ärztin Unispit.": "AA",
    "Wahljahrstudent/in": "WJ",
}


In [70]:
DUTY_CODES = {
    100:  "Besonderes",

    101:  "Tagdienst gelb Oberarzt",
    102:  "Spätdienst Zone IB Oberarzt",
    103:  "Nachtdienst Oberarzt",

    1072: "Tagdienst blau Assistenzarzt",
    113:  "Tagdienst gelb Assistenzarzt",
    117:  "Bürotag",
    119:  "Tagdienst blau Oberarzt",

    123:  "Lehre",
    128:  "Tagdienst Wochenende / Feiertag Oberarzt",
    129:  "Nachtdienst Assistenzarzt",
    134:  "Nachtdienst Wochenende / Feiertag Assistenzarzt",

    165:  "Tagdienst IMC Oberarzt",
    166:  "Spätdienst IMC",
    1704: "Pikett 12h Intervention <30min",
    175:  "Tagdienst Wochenende / Feiertag Assistenzarzt",

    180:  "Nachtdienst Wochenende / Feiertag Oberarzt",
    2461: "Tagdienst IB grün",
    271:  "Spätdienst Intensivstation",
    2834: "KISS",

    3000: "Ferien",
    3025: "Ferien UNI-besoldet / fiktiv",
    3080: "Bildung",
    3081: "Bildung OA",
    3085: "Bildung BG",
    3095: "Bildung intern",
    3096: "Bildung intern OA",

    3110: "Urlaub des anderen Elternteils",
    3120: "Dienstalter",
    3130: "Komp. Überstunden",
    3135: "Komp. Zeitgutschrift",
    3136: "Komp. Zeitgutschrift OA",

    3140: "Ruhetag",
    3145: "Freier Tag",
    3150: "Wunschfrei",

    371:  "Umzug",
    374:  "Auswärtige Sitzung",

    705:  "Forschung OA",
    719:  "Tagdienst Neuro IMC",
    721:  "Tagdienst Zone IMC Viszeral",
    741:  "Forschung AA",

    823:  "S-Dienst",
    826:  "Einführung",
    827:  "Betriebsleitung",
}


In [71]:
DUTY_CLASS = {
    # =========================
    # TAGDIENSTE
    # =========================
    101: "TAGDIENST",        # OA | Tagdienst gelb Oberarzt
    119: "TAGDIENST",        # OA | Tagdienst blau Oberarzt
    128: "TAGDIENST_WE",     # OA | Tagdienst Wochenende / Feiertag Oberarzt
    165: "TAGDIENST_IMC",    # OA | Tagdienst IMC Oberarzt
    2461: "TAGDIENST",       # OA | Tagdienst IB grün
    719: "TAGDIENST_IMC",    # OA | Tagdienst Neuro IMC
    721: "TAGDIENST_IMC",    # OA | Tagdienst Zone IMC Viszeral

    1072: "TAGDIENST",       # AA | Tagdienst blau Assistenzarzt
    113: "TAGDIENST",        # AA | Tagdienst gelb Assistenzarzt
    175: "TAGDIENST_WE",     # AA | Tagdienst Wochenende / Feiertag Assistenzarzt

    # =========================
    # SPÄTDIENSTE
    # =========================
    102: "SPAETDIENST",      # OA | Spätdienst Zone IB
    166: "SPAETDIENST",      # OA | Spätdienst IMC
    271: "SPAETDIENST",      # AA | Spätdienst Intensivstation

    # =========================
    # NACHTDIENSTE
    # =========================
    103: "NACHTDIENST",      # OA | Nachtdienst
    129: "NACHTDIENST",      # AA | Nachtdienst
    134: "NACHTDIENST_WE",   # AA | Nachtdienst WE / Feiertag
    180: "NACHTDIENST_WE",   # OA | Nachtdienst WE / Feiertag

    # =========================
    # NICHT VERFÜGBAR
    # =========================
    3000: "FERIEN",          # ALL | Ferien
    3025: "FERIEN",          # ALL | Ferien UNI / fiktiv
    3140: "RUHETAG",         # ALL | Ruhetag
    3145: "FREI",            # ALL | Freier Tag
    3150: "WUNSCHFREI",      # ALL | Wunschfrei

    371:  "NICHT_VERFUEGBAR",# ALL | Umzug
    374:  "NICHT_VERFUEGBAR",# ALL | Auswärtige Sitzung
    100:  "NICHT_VERFUEGBAR",# ALL | Besonderes

    3120: "NICHT_VERFUEGBAR",# ALL | Dienstalter
    3130: "NICHT_VERFUEGBAR",# ALL | Komp. Überstunden
    3135: "NICHT_VERFUEGBAR",# ALL | Komp. Zeitgutschrift
    3136: "NICHT_VERFUEGBAR",# ALL | Komp. Zeitgutschrift OA
    3110: "NICHT_VERFUEGBAR",# ALL | Urlaub anderer Elternteil

    # =========================
    # BILDUNG / LEHRE
    # =========================
    3080: "BILDUNG",         # ALL | Bildung
    3081: "BILDUNG",         # OA  | Bildung OA
    3085: "BILDUNG",         # BG  | Bildung BG
    3095: "BILDUNG",         # ALL | Bildung intern
    3096: "BILDUNG",         # OA  | Bildung intern OA

    705:  "FORSCHUNG",       # OA  | Forschung OA
    741:  "FORSCHUNG",       # AA  | Forschung AA

    123:  "LEHRE",           # ALL | Lehre

    117:  "Bürotag",         # ALL | Bürotag

    # =========================
    # SPEZIAL / LEITUNG
    # =========================
    1704: "PIKETT",          # LA/CA | Pikett 12h Intervention
    823:  "S_DIENST",        # LA/CA | S-Dienst
    826:  "EINFUEHRUNG",     # AA    | Einführung
    827:  "BETRIEBSLEITUNG", # LA/CA | Betriebsleitung

    # =========================
    # SONSTIG
    # =========================
    2834: "NICHT_VERFUEGBAR",# ALL | KISS
}


In [72]:
def find_day_header_row(df: pd.DataFrame):
    for i in range(len(df)):
        row = df.iloc[i]
        numeric_days = [
            x for x in row
            if isinstance(x, (int, float)) and 1 <= x <= 31
        ]
        if len(numeric_days) >= 20:  # Schwelle: sicher ein Monat
            return i
    raise ValueError("Keine Kopfzeile mit Tagen gefunden")



In [73]:
def clean_person_name(name: str) -> str:
    # entfernt alles in eckigen Klammern inkl. Inhalt
    name = re.sub(r"\[.*?\]", "", name)
    # doppelte Leerzeichen aufräumen
    return re.sub(r"\s+", " ", name).strip()


In [74]:
def load_pep_raw(path: str) -> pd.DataFrame:
    return pd.read_excel(path, header=None)


In [75]:
def parse_pep_sheet(df: pd.DataFrame, year: int, month: int):
    current_role = None
    records = []

    day_header_row = find_day_header_row(df)
    day_row = df.iloc[day_header_row]

    # Spalten mit gültigen Tagen (1–31)
    day_columns = {
        col_idx: int(day)
        for col_idx, day in enumerate(day_row)
        if isinstance(day, (int, float)) and 1 <= day <= 31
    }

    for idx in range(day_header_row + 1, len(df)):
        row = df.iloc[idx]
        first_cell = str(row[0]).strip()

        #  Rollenüberschrift erkennen
        if first_cell in ROLE_MAP:
            current_role = first_cell
            continue

        #  Abbruch bei leeren Bereichen
        if not first_cell or first_cell == "nan":
            continue

        #  Personenzeile
        if current_role and re.search(r"[A-Za-z]", first_cell):
            person_name = clean_person_name(first_cell)
            role_code = ROLE_MAP[current_role]

            for col_idx, day in day_columns.items():
                duty = row[col_idx]

                if pd.isna(duty):
                    continue

                try:
                    duty = int(duty)
                except:
                    continue

                records.append({
                    "person_name": person_name,
                    "role_category": current_role,
                    "role_code": role_code,
                    "date": date(year, month, day),
                    "duty_code": duty,
                    "duty_name": DUTY_CODES.get(duty, "UNBEKANNT"),
                    "duty_class": DUTY_CLASS.get(duty, "UNBEKANNT"),
                })

    return pd.DataFrame(records)


In [76]:
pep_feb = load_pep_raw("2026.02_pep_feb.xlsx")
pep_mar = load_pep_raw("2026.03_pep_märz.xlsx")



In [77]:
df_feb = parse_pep_sheet(pep_feb, 2026, 2)
df_mar = parse_pep_sheet(pep_mar, 2026, 3)

In [78]:
pep_all = pd.concat([df_feb, df_mar], ignore_index=True)


In [79]:
pep_all.head(60)

,person_name,role_category,role_code,date,duty_code,duty_name,duty_class
0,Schefold Joerg Christian,Chefarzt/ärztin Unispital,CA,2026-02-01,3150,Wunschfrei,WUNSCHFREI
1,Schefold Joerg Christian,Chefarzt/ärztin Unispital,CA,2026-02-02,3000,Ferien,FERIEN
2,Schefold Joerg Christian,Chefarzt/ärztin Unispital,CA,2026-02-03,3000,Ferien,FERIEN
3,Schefold Joerg Christian,Chefarzt/ärztin Unispital,CA,2026-02-04,3000,Ferien,FERIEN
4,Schefold Joerg Christian,Chefarzt/ärztin Unispital,CA,2026-02-05,3000,Ferien,FERIEN
5,Schefold Joerg Christian,Chefarzt/ärztin Unispital,CA,2026-02-06,3000,Ferien,FERIEN
6,Schefold Joerg Christian,Chefarzt/ärztin Unispital,CA,2026-02-07,3150,Wunschfrei,WUNSCHFREI
7,Schefold Joerg Christian,Chefarzt/ärztin Unispital,CA,2026-02-08,3150,Wunschfrei,WUNSCHFREI
8,Schefold Joerg Christian,Chefarzt/ärztin Unispital,CA,2026-02-23,827,Betriebsleitung,BETRIEBSLEITUNG
9,Schefold Joerg Christian,Chefarzt/ärztin Unispital,CA,2026-02-24,827,Betriebsleitung,BETRIEBSLEITUNG


# Schedule

##  MONDAYS KIM Sim ICU/IMC

In [80]:
### WHAT IS RULE? # see placeholder here

In [81]:
import random
from datetime import date
import pandas as pd

def mondays_in_month(year: int, month: int):
    dates = pd.date_range(
        start=date(year, month, 1),
        end=pd.Timestamp(year, month, 1) + pd.offsets.MonthEnd(1),
        freq="D",
    )
    return [d.date() for d in dates if d.weekday() == 0]

def random_kimsim_monday(year: int, month: int, base_seed: int = 42):
    rng = random.Random(f"{base_seed}-{year}-{month}")
    mondays = mondays_in_month(year, month)

    if not mondays:
        raise ValueError("No Mondays found")

    return rng.choice(mondays)


In [82]:
def build_kimsim_schedule_random(year: int, base_seed: int = 42) -> pd.DataFrame:
    rows = []

    for month in range(1, 13):
        kimsim_date = random_kimsim_monday(year, month, base_seed)

        for time in ["07:30-11:15", "12:30-16:15"]:
            rows.append({
                "date": kimsim_date,
                "weekday": "MO",
                "time": time,
                "format": "KIMSIM",
                "thema_final": "KimSim ICU oder IMC",
                "verantwortliche": "",
                "selection_method": "random_monday",
            })

    return pd.DataFrame(rows)

kimsim_2026 = build_kimsim_schedule_random(2026, base_seed=2026)
kimsim_2026


,date,weekday,time,format,thema_final,verantwortliche,selection_method
0,2026-01-12,MO,07:30-11:15,KIMSIM,KimSim ICU oder IMC,,random_monday
1,2026-01-12,MO,12:30-16:15,KIMSIM,KimSim ICU oder IMC,,random_monday
2,2026-02-16,MO,07:30-11:15,KIMSIM,KimSim ICU oder IMC,,random_monday
3,2026-02-16,MO,12:30-16:15,KIMSIM,KimSim ICU oder IMC,,random_monday
4,2026-03-02,MO,07:30-11:15,KIMSIM,KimSim ICU oder IMC,,random_monday
5,2026-03-02,MO,12:30-16:15,KIMSIM,KimSim ICU oder IMC,,random_monday
6,2026-04-06,MO,07:30-11:15,KIMSIM,KimSim ICU oder IMC,,random_monday
7,2026-04-06,MO,12:30-16:15,KIMSIM,KimSim ICU oder IMC,,random_monday
8,2026-05-25,MO,07:30-11:15,KIMSIM,KimSim ICU oder IMC,,random_monday
9,2026-05-25,MO,12:30-16:15,KIMSIM,KimSim ICU oder IMC,,random_monday


## MONDAYS Interprof. WB



In [83]:
def generate_mondays(year: int) -> pd.DataFrame:
    dates = pd.date_range(
        start=date(year, 1, 1),
        end=date(year, 12, 31),
        freq="D",
    )

    rows = []
    for d in dates:
        if d.weekday() == 0:  # Monday
            rows.append({
                "date": d.date(),
                "weekday": "MO",
            })

    return pd.DataFrame(rows)

mondays_2026 = generate_mondays(2026)
mondays_2026.head()


,date,weekday
0,2026-01-05,MO
1,2026-01-12,MO
2,2026-01-19,MO
3,2026-01-26,MO
4,2026-02-02,MO


In [84]:
def build_interprof_wb_schedule(mondays_df):
    df = mondays_df.copy()

    df["time"] = "14:45-15:30"
    df["format"] = "INTERPROF_WB"
    df["verantwortliche"] = ""  # intentionally empty
    df["thema_final"] = "Interprofessionelle Weiterbildung"

    return df[
        [
            "date",
            "weekday",
            "time",
            "format",
            "verantwortliche",
            "thema_final",
        ]
    ]


In [85]:
interprof_wb_2026 = build_interprof_wb_schedule(mondays_2026)
interprof_wb_2026.head(10)


,date,weekday,time,format,verantwortliche,thema_final
0,2026-01-05,MO,14:45-15:30,INTERPROF_WB,,Interprofessionelle Weiterbildung
1,2026-01-12,MO,14:45-15:30,INTERPROF_WB,,Interprofessionelle Weiterbildung
2,2026-01-19,MO,14:45-15:30,INTERPROF_WB,,Interprofessionelle Weiterbildung
3,2026-01-26,MO,14:45-15:30,INTERPROF_WB,,Interprofessionelle Weiterbildung
4,2026-02-02,MO,14:45-15:30,INTERPROF_WB,,Interprofessionelle Weiterbildung
5,2026-02-09,MO,14:45-15:30,INTERPROF_WB,,Interprofessionelle Weiterbildung
6,2026-02-16,MO,14:45-15:30,INTERPROF_WB,,Interprofessionelle Weiterbildung
7,2026-02-23,MO,14:45-15:30,INTERPROF_WB,,Interprofessionelle Weiterbildung
8,2026-03-02,MO,14:45-15:30,INTERPROF_WB,,Interprofessionelle Weiterbildung
9,2026-03-09,MO,14:45-15:30,INTERPROF_WB,,Interprofessionelle Weiterbildung


## TUESDAY “Peer-Teaching” oder “Case of the day (COD)” oder “Physiotalk”

### Days

first tueday - senior physician cod

second tuesday - physio talk. after that alternating monthly/bimonthly

rest of free spaces - alternate between cod and peer teaching

In [86]:
def generate_tuesday_morning_slots(year: int) -> pd.DataFrame:
    dates = pd.date_range(
        start=date(year, 1, 1),
        end=date(year, 12, 31),
        freq="D",
    )

    rows = []
    for d in dates:
        if d.weekday() == 1:  # Tuesday
            rows.append({
                "date": d.date(),
                "weekday": "DI",
                "time": "11:30-11:45",
                "format": "DI_MORNING",
            })

    return pd.DataFrame(rows)


In [87]:
tuesday_morning_2026 = generate_tuesday_morning_slots(2026)
tuesday_morning_2026.head()


,date,weekday,time,format
0,2026-01-06,DI,11:30-11:45,DI_MORNING
1,2026-01-13,DI,11:30-11:45,DI_MORNING
2,2026-01-20,DI,11:30-11:45,DI_MORNING
3,2026-01-27,DI,11:30-11:45,DI_MORNING
4,2026-02-03,DI,11:30-11:45,DI_MORNING


In [88]:
def add_month_metadata(df):
    df = df.copy()
    df["month"] = df["date"].apply(lambda d: d.month)
    df["week_in_month"] = df["date"].apply(lambda d: (d.day - 1) // 7 + 1)
    return df


In [89]:
tuesday_morning_2026 = add_month_metadata(tuesday_morning_2026)
tuesday_morning_2026.head(10)


,date,weekday,time,format,month,week_in_month
0,2026-01-06,DI,11:30-11:45,DI_MORNING,1,1
1,2026-01-13,DI,11:30-11:45,DI_MORNING,1,2
2,2026-01-20,DI,11:30-11:45,DI_MORNING,1,3
3,2026-01-27,DI,11:30-11:45,DI_MORNING,1,4
4,2026-02-03,DI,11:30-11:45,DI_MORNING,2,1
5,2026-02-10,DI,11:30-11:45,DI_MORNING,2,2
6,2026-02-17,DI,11:30-11:45,DI_MORNING,2,3
7,2026-02-24,DI,11:30-11:45,DI_MORNING,2,4
8,2026-03-03,DI,11:30-11:45,DI_MORNING,3,1
9,2026-03-10,DI,11:30-11:45,DI_MORNING,3,2


In [90]:
DI_MORNING_CONFIG = {
    "physiotalk_start_month": 1,      # January
    "physiotalk_interval": 2,          # every 2 months
    "backbone_cycle": ["PEER_TEACHING", "COD"],
}

def is_physiotalk_month(month, config):
    return (
        (month - config["physiotalk_start_month"]) %
        config["physiotalk_interval"]
    ) == 0



def month_index(d):
    return d.year * 12 + d.month


In [91]:
def assign_di_morning_formats_final(df, config):
    df = df.sort_values("date").copy()

    backbone_idx = 0
    formats = []

    for _, row in df.iterrows():
        week = row["week_in_month"]
        month = row["month"]

        # 1️⃣ First Tuesday → COD (senior)
        if week == 1:
            formats.append("COD_SENIOR")
            continue

        # 2️⃣ Second Tuesday → Physiotalk (only in Physiotalk months)
        if week == 2 and is_physiotalk_month(month, config):
            formats.append("PHYSIOTALK")
            continue

        # 3️⃣ Backbone alternation (continues across months)
        fmt = config["backbone_cycle"][backbone_idx]
        formats.append(fmt)
        backbone_idx = (backbone_idx + 1) % len(config["backbone_cycle"])

    df["subformat"] = formats
    return df


In [92]:
tuesday_morning_2026 = assign_di_morning_formats_final(
    tuesday_morning_2026,
    DI_MORNING_CONFIG,
)

tuesday_morning_2026.head(12)


,date,weekday,time,format,month,week_in_month,subformat
0,2026-01-06,DI,11:30-11:45,DI_MORNING,1,1,COD_SENIOR
1,2026-01-13,DI,11:30-11:45,DI_MORNING,1,2,PHYSIOTALK
2,2026-01-20,DI,11:30-11:45,DI_MORNING,1,3,PEER_TEACHING
3,2026-01-27,DI,11:30-11:45,DI_MORNING,1,4,COD
4,2026-02-03,DI,11:30-11:45,DI_MORNING,2,1,COD_SENIOR
5,2026-02-10,DI,11:30-11:45,DI_MORNING,2,2,PEER_TEACHING
6,2026-02-17,DI,11:30-11:45,DI_MORNING,2,3,COD
7,2026-02-24,DI,11:30-11:45,DI_MORNING,2,4,PEER_TEACHING
8,2026-03-03,DI,11:30-11:45,DI_MORNING,3,1,COD_SENIOR
9,2026-03-10,DI,11:30-11:45,DI_MORNING,3,2,PHYSIOTALK


In [93]:
pep_all.head()

,person_name,role_category,role_code,date,duty_code,duty_name,duty_class
0,Schefold Joerg Christian,Chefarzt/ärztin Unispital,CA,2026-02-01,3150,Wunschfrei,WUNSCHFREI
1,Schefold Joerg Christian,Chefarzt/ärztin Unispital,CA,2026-02-02,3000,Ferien,FERIEN
2,Schefold Joerg Christian,Chefarzt/ärztin Unispital,CA,2026-02-03,3000,Ferien,FERIEN
3,Schefold Joerg Christian,Chefarzt/ärztin Unispital,CA,2026-02-04,3000,Ferien,FERIEN
4,Schefold Joerg Christian,Chefarzt/ärztin Unispital,CA,2026-02-05,3000,Ferien,FERIEN


### Assign Senior

In [94]:
# S Dienst if available, otherwise Büro or Forschung, otherwise Spätdienst?
SENIOR_DUTY_PRIORITY = [
    # 1️ Ideal
    {"S_DIENST"},

    # ??
    {"BETRIEBSLEITUNG", "TAGDIENST", "TAGDIENST_IMC"},

    #
    {"BÜROTAG","FORSCHUNG"},

    #
    {"SPAETDIENST"},

    #  Fallback ?
    { "PIKETT"},
]


In [95]:
SENIOR_ROLES = ["SFA_I", "LA", "SCA", "CA"]


In [96]:
pep_df = pep_all.copy()
pep_df.head()



,person_name,role_category,role_code,date,duty_code,duty_name,duty_class
0,Schefold Joerg Christian,Chefarzt/ärztin Unispital,CA,2026-02-01,3150,Wunschfrei,WUNSCHFREI
1,Schefold Joerg Christian,Chefarzt/ärztin Unispital,CA,2026-02-02,3000,Ferien,FERIEN
2,Schefold Joerg Christian,Chefarzt/ärztin Unispital,CA,2026-02-03,3000,Ferien,FERIEN
3,Schefold Joerg Christian,Chefarzt/ärztin Unispital,CA,2026-02-04,3000,Ferien,FERIEN
4,Schefold Joerg Christian,Chefarzt/ärztin Unispital,CA,2026-02-05,3000,Ferien,FERIEN


In [97]:
class SeniorRotator:
    def __init__(self):
        self.last_idx = {}

    def pick(self, seniors_df, pep_df, date, key="COD_SENIOR"):
        """
        seniors_df: dataframe with at least ['person_name']
        pep_df: full pep table
        date: datetime.date
        """

        if seniors_df.empty:
            return None

        names = seniors_df["person_name"].tolist()

        # round-robin
        idx = self.last_idx.get(key, -1)
        idx = (idx + 1) % len(names)
        self.last_idx[key] = idx

        chosen_name = names[idx]

        # enrich with duty info from pep_df (for debugging / traceability)
        duty_info = pep_df[
            (pep_df["date"] == date)
            & (pep_df["person_name"] == chosen_name)
        ][["duty_class", "duty_name"]].head(1)

        if not duty_info.empty:
            duty_class = duty_info.iloc[0]["duty_class"]
            duty_name = duty_info.iloc[0]["duty_name"]
            return f"{chosen_name} [{duty_class} | {duty_name}]"

        # fallback (should rarely happen)
        return chosen_name


In [98]:
def pick_senior_by_duty_priority(
    pep_df,
    date,
    senior_rotator,
    rotator_key="COD_SENIOR",
):
    seniors_today = pep_df[
        (pep_df["date"] == date)
        & (pep_df["role_code"].isin(SENIOR_ROLES))
        & (~pep_df["duty_class"].isin({"FERIEN", "RUHETAG", "FREI"}))
    ][["person_name", "role_code", "duty_class"]]

    if seniors_today.empty:
        return None

    for duty_group in SENIOR_DUTY_PRIORITY:
        candidates = seniors_today[
            seniors_today["duty_class"].isin(duty_group)
        ]

        if not candidates.empty:
            return senior_rotator.pick(
                seniors_df=candidates,
                pep_df=pep_df,
                date=date,
                key=rotator_key,
            )

    return None


In [99]:
senior_rotator = SeniorRotator()


def assign_senior_for_cod(row, pep_df, senior_rotator):
    if row["subformat"] != "COD_SENIOR":
        return None

    person = pick_senior_by_duty_priority(
        pep_df=pep_df,
        date=row["date"],
        senior_rotator=senior_rotator,
        rotator_key="COD_SENIOR",
    )

    return person or "***KEIN SENIOR VERFÜGBAR***"




In [100]:
tuesday_morning_2026["verantwortliche"] = tuesday_morning_2026.apply(
    lambda r: assign_senior_for_cod(r, pep_df, senior_rotator),
    axis=1,
)

tuesday_morning_2026.head(16)

,date,weekday,time,format,month,week_in_month,subformat,verantwortliche
0,2026-01-06,DI,11:30-11:45,DI_MORNING,1,1,COD_SENIOR,***KEIN SENIOR VERFÜGBAR***
1,2026-01-13,DI,11:30-11:45,DI_MORNING,1,2,PHYSIOTALK,None
2,2026-01-20,DI,11:30-11:45,DI_MORNING,1,3,PEER_TEACHING,None
3,2026-01-27,DI,11:30-11:45,DI_MORNING,1,4,COD,None
4,2026-02-03,DI,11:30-11:45,DI_MORNING,2,1,COD_SENIOR,Que Yok-Ai [S_DIENST | S-Dienst]
5,2026-02-10,DI,11:30-11:45,DI_MORNING,2,2,PEER_TEACHING,None
6,2026-02-17,DI,11:30-11:45,DI_MORNING,2,3,COD,None
7,2026-02-24,DI,11:30-11:45,DI_MORNING,2,4,PEER_TEACHING,None
8,2026-03-03,DI,11:30-11:45,DI_MORNING,3,1,COD_SENIOR,Ludwig Roger 2 [S_DIENST | S-Dienst]
9,2026-03-10,DI,11:30-11:45,DI_MORNING,3,2,PHYSIOTALK,None


### Assign AA

In [101]:
AA_ALLOWED_DUTY_CODES = {
    113,   # Tagdienst gelb Assistenzarzt
    1072,  # Tagdienst blau Assistenzarzt
    719,   # Tagdienst Neuro IMC
}


In [102]:
def available_aa_by_code(pep_df, date):
    return pep_df[
        (pep_df["date"] == date)
        & (pep_df["role_code"] == "AA")
        & (pep_df["duty_code"].isin({113, 1072, 719}))
        & (~pep_df["duty_class"].isin({"FERIEN", "RUHETAG", "FREI"}))
    ][["person_name"]]


In [103]:
class AARotator:
    def __init__(self):
        self.last_idx = {}

    def pick(self, df, key="AA"):
        if df.empty:
            return None

        names = sorted(df["person_name"].unique().tolist())

        idx = self.last_idx.get(key, -1)
        idx = (idx + 1) % len(names)

        self.last_idx[key] = idx
        return names[idx]


In [104]:
aa_rotator = AARotator()

def assign_aa_for_di_morning(row, pep_df, aa_rotator):
    if row["subformat"] == "COD_SENIOR":
        return None

    aas = available_aa_by_code(
        pep_df=pep_df,
        date=row["date"],
    )

    person = aa_rotator.pick(aas)

    if person is None:
        return "***KEIN AA TAGDIENST VERFÜGBAR***"

    return person


In [105]:
mask_needs_aa = (
    tuesday_morning_2026["verantwortliche"].isna()
)

tuesday_morning_2026.loc[mask_needs_aa, "verantwortliche"] = (
    tuesday_morning_2026.loc[mask_needs_aa]
    .apply(
        lambda r: assign_aa_for_di_morning(r, pep_all, aa_rotator),
        axis=1,
    )
)


In [106]:
# fairness checker
(
    tuesday_morning_2026
    .query("verantwortliche.notnull()")
    .query("verantwortliche.str.startswith('***') == False")
    .groupby("verantwortliche")
    .size()
    .sort_values(ascending=False)
)


,0
verantwortliche,
Studer Fabian,2
Conrad Carl David Felix,1
Kyriazi Maria,1
Jungi Noël,1
Ludwig Roger 2 [S_DIENST | S-Dienst],1
Que Yok-Ai [S_DIENST | S-Dienst],1
Rösli David Luca,1
Sula Martin,1


In [107]:
tuesday_morning_2026[
    tuesday_morning_2026["verantwortliche"]
    == tuesday_morning_2026["verantwortliche"].shift()
][["date", "verantwortliche"]]


,date,verantwortliche
2,2026-01-20,***KEIN AA TAGDIENST VERFÜGBAR***
3,2026-01-27,***KEIN AA TAGDIENST VERFÜGBAR***
15,2026-04-21,***KEIN AA TAGDIENST VERFÜGBAR***
16,2026-04-28,***KEIN AA TAGDIENST VERFÜGBAR***
19,2026-05-19,***KEIN AA TAGDIENST VERFÜGBAR***
20,2026-05-26,***KEIN AA TAGDIENST VERFÜGBAR***
23,2026-06-16,***KEIN AA TAGDIENST VERFÜGBAR***
24,2026-06-23,***KEIN AA TAGDIENST VERFÜGBAR***
25,2026-06-30,***KEIN AA TAGDIENST VERFÜGBAR***
28,2026-07-21,***KEIN AA TAGDIENST VERFÜGBAR***


In [108]:
tuesday_morning_2026.head(16)

,date,weekday,time,format,month,week_in_month,subformat,verantwortliche
0,2026-01-06,DI,11:30-11:45,DI_MORNING,1,1,COD_SENIOR,***KEIN SENIOR VERFÜGBAR***
1,2026-01-13,DI,11:30-11:45,DI_MORNING,1,2,PHYSIOTALK,***KEIN AA TAGDIENST VERFÜGBAR***
2,2026-01-20,DI,11:30-11:45,DI_MORNING,1,3,PEER_TEACHING,***KEIN AA TAGDIENST VERFÜGBAR***
3,2026-01-27,DI,11:30-11:45,DI_MORNING,1,4,COD,***KEIN AA TAGDIENST VERFÜGBAR***
4,2026-02-03,DI,11:30-11:45,DI_MORNING,2,1,COD_SENIOR,Que Yok-Ai [S_DIENST | S-Dienst]
5,2026-02-10,DI,11:30-11:45,DI_MORNING,2,2,PEER_TEACHING,Conrad Carl David Felix
6,2026-02-17,DI,11:30-11:45,DI_MORNING,2,3,COD,Jungi Noël
7,2026-02-24,DI,11:30-11:45,DI_MORNING,2,4,PEER_TEACHING,Kyriazi Maria
8,2026-03-03,DI,11:30-11:45,DI_MORNING,3,1,COD_SENIOR,Ludwig Roger 2 [S_DIENST | S-Dienst]
9,2026-03-10,DI,11:30-11:45,DI_MORNING,3,2,PHYSIOTALK,Studer Fabian


### TOPICS COD and Peer Teaching

### TOPICS Physiotalk

In [109]:
def load_physiotalk_articles(path: str):
    df = pd.read_excel(path)

    df = df.rename(columns={
        "Nr.": "nr",
        "Titel": "titel",
    })

    df = df.dropna(subset=["titel"])
    df = df.sort_values("nr")

    return df.to_dict(orient="records")

physiotalk_articles = load_physiotalk_articles("2026.01_Physiotalk_Artikelübersicht.xlsx")
physiotalk_articles[:5]


[{'nr': 1.0,
  'titel': 'Intrinsic (or auto-) PEEP during controlled mechanical ventilation',
  'Jahr': nan},
 {'nr': 2.0,
  'titel': 'Intrinsic (or auto-) positive end-expiratory pressure during spontaneous or assisted ventilation',
  'Jahr': nan},
 {'nr': 3.0, 'titel': 'Work of breathing', 'Jahr': nan},
 {'nr': 4.0,
  'titel': 'Interpretation of airway pressure waveforms',
  'Jahr': nan},
 {'nr': 5.0,
  'titel': 'Measurement of respiratory system resistance during mechanical ventilation',
  'Jahr': nan}]

In [110]:
class PhysiotalkRotator:
    def __init__(self, articles):
        if not articles:
            raise ValueError("No Physiotalk articles provided")
        self.articles = articles
        self.idx = 0

    def next(self):
        article = self.articles[self.idx]
        self.idx = (self.idx + 1) % len(self.articles)
        return article

physiotalk_rotator = PhysiotalkRotator(physiotalk_articles)


In [111]:
def assign_physiotalk_topic(row, rotator):
    if row["subformat"] != "PHYSIOTALK":
        return None

    article = rotator.next()
    return f"Physiotalk – {article['nr']}. {article['titel']}"


In [112]:
tuesday_morning_2026 = tuesday_morning_2026.copy()

tuesday_morning_2026["thema"] = tuesday_morning_2026.apply(
    lambda r: assign_physiotalk_topic(r, physiotalk_rotator),
    axis=1,
)


In [113]:
tuesday_morning_2026[
    tuesday_morning_2026["subformat"] == "PHYSIOTALK"
][["date", "subformat", "thema"]]


,date,subformat,thema
1,2026-01-13,PHYSIOTALK,Physiotalk – 1.0. Intrinsic (or auto-) PEEP du...
9,2026-03-10,PHYSIOTALK,Physiotalk – 2.0. Intrinsic (or auto-) positiv...
18,2026-05-12,PHYSIOTALK,Physiotalk – 3.0. Work of breathing
27,2026-07-14,PHYSIOTALK,Physiotalk – 4.0. Interpretation of airway pre...
35,2026-09-08,PHYSIOTALK,Physiotalk – 5.0. Measurement of respiratory s...
44,2026-11-10,PHYSIOTALK,Physiotalk – 6.0. Understanding wasted-ineffec...


In [114]:
tuesday_morning_2026.head(16)

,date,weekday,time,format,month,week_in_month,subformat,verantwortliche,thema
0,2026-01-06,DI,11:30-11:45,DI_MORNING,1,1,COD_SENIOR,***KEIN SENIOR VERFÜGBAR***,None
1,2026-01-13,DI,11:30-11:45,DI_MORNING,1,2,PHYSIOTALK,***KEIN AA TAGDIENST VERFÜGBAR***,Physiotalk – 1.0. Intrinsic (or auto-) PEEP du...
2,2026-01-20,DI,11:30-11:45,DI_MORNING,1,3,PEER_TEACHING,***KEIN AA TAGDIENST VERFÜGBAR***,None
3,2026-01-27,DI,11:30-11:45,DI_MORNING,1,4,COD,***KEIN AA TAGDIENST VERFÜGBAR***,None
4,2026-02-03,DI,11:30-11:45,DI_MORNING,2,1,COD_SENIOR,Que Yok-Ai [S_DIENST | S-Dienst],None
5,2026-02-10,DI,11:30-11:45,DI_MORNING,2,2,PEER_TEACHING,Conrad Carl David Felix,None
6,2026-02-17,DI,11:30-11:45,DI_MORNING,2,3,COD,Jungi Noël,None
7,2026-02-24,DI,11:30-11:45,DI_MORNING,2,4,PEER_TEACHING,Kyriazi Maria,None
8,2026-03-03,DI,11:30-11:45,DI_MORNING,3,1,COD_SENIOR,Ludwig Roger 2 [S_DIENST | S-Dienst],None
9,2026-03-10,DI,11:30-11:45,DI_MORNING,3,2,PHYSIOTALK,Studer Fabian,Physiotalk – 2.0. Intrinsic (or auto-) positiv...


## TUESDAY interprofessionelles IMC-Update

In [115]:
def load_imc_update_excel(path: str) -> pd.DataFrame:
    df = pd.read_excel(path)

    df = df.rename(columns={
        "Datum": "date",
        "Verantwortliche": "verantwortliche",
        "Thema": "thema",
        "Ort": "ort",
        "Definitiv planbar": "definitiv_planbar",
    })

    df["date"] = pd.to_datetime(
        df["date"],
        format="%d.%m.%Y",
        errors="raise"
    ).dt.date

    df["definitiv_planbar"] = (
        df["definitiv_planbar"]
        .fillna("")
        .str.lower()
        .str.strip()
    )

    return df


In [116]:
def is_second_tuesday(d):
    return d.weekday() == 1 and 8 <= d.day <= 14

def validate_imc_update_dates(df):
    invalid = df[~df["date"].apply(is_second_tuesday)]
    if not invalid.empty:
        raise ValueError(
            "IMC Update contains non–2nd Tuesday dates:\n"
            + invalid[["date"]].to_string(index=False)
        )


In [117]:
imc_df = load_imc_update_excel("IMC_Updates.xlsx")
validate_imc_update_dates(imc_df)


In [118]:
def build_imc_update_schedule(imc_df):
    df = imc_df.copy()

    df["weekday"] = "DI"
    df["time"] = "15:00-15:45"
    df["format"] = "IMC_UPDATE"

    def mark_uncertain(text, is_definitiv):
        if is_definitiv:
            return text
        return f"***{text}"

    def build_fields(row):
        is_definitiv = row["definitiv_planbar"] == "ja"

        verantwortliche = row["verantwortliche"]
        thema = row["thema"]

        # Sommerpause explizit durchreichen
        if "sommerpause" in str(verantwortliche).lower():
            return verantwortliche, thema

        verantwortliche = mark_uncertain(verantwortliche, is_definitiv)
        thema = mark_uncertain(thema, is_definitiv)

        return verantwortliche, thema

    df[["verantwortliche_final", "thema_final"]] = df.apply(
        build_fields, axis=1, result_type="expand"
    )

    return df[
        [
            "date",
            "weekday",
            "time",
            "format",
            "verantwortliche_final",
            "thema_final",
            "ort",
            "definitiv_planbar",
        ]
    ]


In [119]:
imc_update_2026 = build_imc_update_schedule(imc_df)
imc_update_2026


,date,weekday,time,format,verantwortliche_final,thema_final,ort,definitiv_planbar
0,2026-01-13,DI,15:00-15:45,IMC_UPDATE,Marie-Noëlle Kronig,Hämato-onkologische Erkrankungen: Chemotherapi...,ASH E245,ja
1,2026-02-10,DI,15:00-15:45,IMC_UPDATE,Carmen Pfortmüller,Kreislauf und relevante Medikamente auf der IMC,ASH E245,ja
2,2026-03-10,DI,15:00-15:45,IMC_UPDATE,Externe Firma,Aggressionsmanagement auf der IMC,ASH E245,ja
3,2026-04-14,DI,15:00-15:45,IMC_UPDATE,Manuel Kindler,Ateminsuffizienz und Tracheotomie Management,ASH E245,ja
4,2026-05-12,DI,15:00-15:45,IMC_UPDATE,Florian Setzer,Herzchirurgische Patienten auf der IMC – häufi...,ASH E245,ja
5,2026-06-09,DI,15:00-15:45,IMC_UPDATE,***Robert Hoepner,***Infektiöse und nicht-infektiöse Encephaliti...,ASH E245,
6,2026-07-14,DI,15:00-15:45,IMC_UPDATE,(Sommerpause),(kein IMC Update),NaN,


## TUESDAY Teaching Tuesday


In [120]:
def load_teaching_tuesday_excel(path: str) -> pd.DataFrame:
    df = pd.read_excel(path)

    df = df.rename(columns={
        "Datum": "date",
        "Referent(en)": "referent",
        "Thema": "thema",
        "Ort": "ort",
        "Definitiv planbar": "definitiv_planbar",
    })

    df["date"] = pd.to_datetime(df["date"],format="%d.%m.%Y",errors="raise").dt.date


    # normalize flags
    df["definitiv_planbar"] = (
        df["definitiv_planbar"]
        .fillna("")
        .str.lower()
        .str.strip()
    )

    return df


In [121]:
teaching_df = load_teaching_tuesday_excel("Teaching_Tuesday.xlsx")
teaching_df.head()

,date,referent,thema,ort,definitiv_planbar
0,2026-01-06,Julian Lippert,Wie entwickeln sich unsere neuro(chirurgischen...,OPO E 218,ja
1,2026-02-03,Anna Messmer; Marie-Noelle Kronig,Update CAR-T,OPO E 218,ja
2,2026-03-03,Martin Faltys,KI in der Intensivmedizin,INO E 218,ja
3,2026-04-07,Martha Veit,Herztransplantation,INO E 218,ja
4,2026-05-05,Yves Borbely (angefragt),noch offen,INO E 218,


In [122]:
def validate_teaching_tuesday_dates(df):
    invalid = df[df["date"].apply(lambda d: d.weekday() != 1)]  # 1 = Tuesday

    if not invalid.empty:
        raise ValueError(
            "Teaching Tuesday Excel contains non-Tuesday dates:\n"
            + invalid[["date"]].to_string(index=False)
        )


In [123]:
validate_teaching_tuesday_dates(teaching_df)
# good if empty

In [124]:
def build_teaching_tuesday_schedule(teaching_df):
    df = teaching_df.copy()

    df["weekday"] = "DI"
    df["time"] = "17:30-18:30"
    df["format"] = "TEACHING_TUESDAY"
    df["verantwortliche"] = "Nadja Schai"

    def build_topic(row):
        if row["thema"] == "noch offen":
            return "Teaching Tuesday – Thema offen"
        if row["referent"] != "noch offen":
            return f"{row['thema']} – {row['referent']}"
        return row["thema"]

    df["thema_final"] = df.apply(build_topic, axis=1)

    return df[
        [
            "date",
            "weekday",
            "time",
            "format",
            "verantwortliche",
            "thema_final",
            "ort",
            "definitiv_planbar",
        ]
    ]


In [125]:
teaching_tuesday_2026 = build_teaching_tuesday_schedule(teaching_df)
teaching_tuesday_2026


,date,weekday,time,format,verantwortliche,thema_final,ort,definitiv_planbar
0,2026-01-06,DI,17:30-18:30,TEACHING_TUESDAY,Nadja Schai,Wie entwickeln sich unsere neuro(chirurgischen...,OPO E 218,ja
1,2026-02-03,DI,17:30-18:30,TEACHING_TUESDAY,Nadja Schai,Update CAR-T – Anna Messmer; Marie-Noelle Kronig,OPO E 218,ja
2,2026-03-03,DI,17:30-18:30,TEACHING_TUESDAY,Nadja Schai,KI in der Intensivmedizin – Martin Faltys,INO E 218,ja
3,2026-04-07,DI,17:30-18:30,TEACHING_TUESDAY,Nadja Schai,Herztransplantation – Martha Veit,INO E 218,ja
4,2026-05-05,DI,17:30-18:30,TEACHING_TUESDAY,Nadja Schai,Teaching Tuesday – Thema offen,INO E 218,
5,2026-06-02,DI,17:30-18:30,TEACHING_TUESDAY,Nadja Schai,Teaching Tuesday – Thema offen,INO E 218,
6,2026-07-07,DI,17:30-18:30,TEACHING_TUESDAY,Nadja Schai,Teaching Tuesday – Thema offen,INO E 218,
7,2026-08-04,DI,17:30-18:30,TEACHING_TUESDAY,Nadja Schai,Teaching Tuesday – Thema offen,INO E 218,
8,2026-09-01,DI,17:30-18:30,TEACHING_TUESDAY,Nadja Schai,Teaching Tuesday – Thema offen,INO E 218,
9,2026-10-06,DI,17:30-18:30,TEACHING_TUESDAY,Nadja Schai,Teaching Tuesday – Thema offen,INO E 218,


In [126]:
teaching_tuesday_2026.head()

,date,weekday,time,format,verantwortliche,thema_final,ort,definitiv_planbar
0,2026-01-06,DI,17:30-18:30,TEACHING_TUESDAY,Nadja Schai,Wie entwickeln sich unsere neuro(chirurgischen...,OPO E 218,ja
1,2026-02-03,DI,17:30-18:30,TEACHING_TUESDAY,Nadja Schai,Update CAR-T – Anna Messmer; Marie-Noelle Kronig,OPO E 218,ja
2,2026-03-03,DI,17:30-18:30,TEACHING_TUESDAY,Nadja Schai,KI in der Intensivmedizin – Martin Faltys,INO E 218,ja
3,2026-04-07,DI,17:30-18:30,TEACHING_TUESDAY,Nadja Schai,Herztransplantation – Martha Veit,INO E 218,ja
4,2026-05-05,DI,17:30-18:30,TEACHING_TUESDAY,Nadja Schai,Teaching Tuesday – Thema offen,INO E 218,


## WEDNESDAY Mittwochskurrikulum


DONT UNDERSTAND YET HOW IT WORKS

In [127]:
def generate_wednesdays(year: int) -> pd.DataFrame:
    dates = pd.date_range(
        start=date(year, 1, 1),
        end=date(year, 12, 31),
        freq="D",
    )

    return pd.DataFrame([
        {
            "date": d.date(),
            "weekday": "MI",
            "time": "14:30-15:15",
            "format": "MITTWOCH_CURRICULUM",
        }
        for d in dates if d.weekday() == 2
    ])




In [128]:
WEDNESDAY_ROLES = {"SOA", "OA_I", "OA_II", "SFA_II"}

def available_wednesday_people(pep_df, date):
    return pep_df[
        (pep_df["date"] == date)
        & (pep_df["role_code"].isin(WEDNESDAY_ROLES))
        & (pep_df["duty_class"] == "SPAETDIENST")
    ][["person_name", "role_code"]]



In [129]:
def load_wednesday_topics(path: str):
    df = pd.read_excel(path)
    return df["Thema"].dropna().tolist()


In [130]:
def build_wednesday_placeholder(year: int) -> pd.DataFrame:
    dates = pd.date_range(
        start=date(year, 1, 1),
        end=date(year, 12, 31),
        freq="D",
    )

    return pd.DataFrame([
        {
            "date": d.date(),
            "weekday": "MI",
            "time": "14:30-15:15",
            "format": "MITTWOCH_CURRICULUM",
            "verantwortliche": "***TBD (Mittwochskurrikulum)***",
            "thema_final": "***Thema folgt***",
        }
        for d in dates if d.weekday() == 2
    ])

wednesday_2026 = build_wednesday_placeholder(2026)



## THURSDAY Interprofessionelle Therapieplanung

In [131]:
def generate_thursdays(year: int) -> pd.DataFrame:
    dates = pd.date_range(
        start=date(year, 1, 1),
        end=date(year, 12, 31),
        freq="D",
    )

    return pd.DataFrame([
        {
            "date": d.date(),
            "weekday": "DO",
            "time": "13:00-13:45",
            "format": "THERAPIEPLANUNG",
            "verantwortliche": "Fall-führende Ärzteschaft",
            "thema": "Interprofessionelle Therapieplanung",
        }
        for d in dates if d.weekday() == 3
    ])


thursdays_2026 = generate_thursdays(2026)
thursdays_2026.head(40)


,date,weekday,time,format,verantwortliche,thema
0,2026-01-01,DO,13:00-13:45,THERAPIEPLANUNG,Fall-führende Ärzteschaft,Interprofessionelle Therapieplanung
1,2026-01-08,DO,13:00-13:45,THERAPIEPLANUNG,Fall-führende Ärzteschaft,Interprofessionelle Therapieplanung
2,2026-01-15,DO,13:00-13:45,THERAPIEPLANUNG,Fall-führende Ärzteschaft,Interprofessionelle Therapieplanung
3,2026-01-22,DO,13:00-13:45,THERAPIEPLANUNG,Fall-führende Ärzteschaft,Interprofessionelle Therapieplanung
4,2026-01-29,DO,13:00-13:45,THERAPIEPLANUNG,Fall-führende Ärzteschaft,Interprofessionelle Therapieplanung
5,2026-02-05,DO,13:00-13:45,THERAPIEPLANUNG,Fall-führende Ärzteschaft,Interprofessionelle Therapieplanung
6,2026-02-12,DO,13:00-13:45,THERAPIEPLANUNG,Fall-führende Ärzteschaft,Interprofessionelle Therapieplanung
7,2026-02-19,DO,13:00-13:45,THERAPIEPLANUNG,Fall-führende Ärzteschaft,Interprofessionelle Therapieplanung
8,2026-02-26,DO,13:00-13:45,THERAPIEPLANUNG,Fall-führende Ärzteschaft,Interprofessionelle Therapieplanung
9,2026-03-05,DO,13:00-13:45,THERAPIEPLANUNG,Fall-führende Ärzteschaft,Interprofessionelle Therapieplanung


## FRIDAY  “Bedside Teaching Infektiologie» oder «Journal Club»


In [132]:
### NOT QUITE RIGHT CAUSE ONLY IF CHRSITINE IS HERE BUT I DONT HAVE HER PLAN?

In [133]:
def generate_fridays(year: int) -> pd.DataFrame:
    dates = pd.date_range(
        start=date(year, 1, 1),
        end=date(year, 12, 31),
        freq="D",
    )

    rows = []
    toggle = True

    for d in dates:
        if d.weekday() == 4:
            rows.append({
                "date": d.date(),
                "weekday": "FR",
                "time": "14:45-15:15",
                "format": "FRIDAY_TEACHING",
                "subformat": (
                    "BEDSIDE_INFEKTIOLOGIE" if toggle else "JOURNAL_CLUB"
                ),
            })
            toggle = not toggle

    return pd.DataFrame(rows)


In [134]:
def build_friday_placeholder(year: int) -> pd.DataFrame:
    dates = pd.date_range(
        start=date(year, 1, 1),
        end=date(year, 12, 31),
        freq="D",
    )

    rows = []
    toggle = True

    for d in dates:
        if d.weekday() == 4:  # Friday
            rows.append({
                "date": d.date(),
                "weekday": "FR",
                "time": "14:45-15:15",
                "format": "FRIDAY_TEACHING",
                "subformat": (
                    "BEDSIDE_INFEKTIOLOGIE"
                    if toggle else
                    "JOURNAL_CLUB"
                ),
                "verantwortliche": "***TBD (abhängig von Präsenz)***",
                "thema_final": "***Thema folgt***",
            })
            toggle = not toggle

    return pd.DataFrame(rows)
fridays_2026 = build_friday_placeholder(2026)
fridays_2026.head(40)

,date,weekday,time,format,subformat,verantwortliche,thema_final
0,2026-01-02,FR,14:45-15:15,FRIDAY_TEACHING,BEDSIDE_INFEKTIOLOGIE,***TBD (abhängig von Präsenz)***,***Thema folgt***
1,2026-01-09,FR,14:45-15:15,FRIDAY_TEACHING,JOURNAL_CLUB,***TBD (abhängig von Präsenz)***,***Thema folgt***
2,2026-01-16,FR,14:45-15:15,FRIDAY_TEACHING,BEDSIDE_INFEKTIOLOGIE,***TBD (abhängig von Präsenz)***,***Thema folgt***
3,2026-01-23,FR,14:45-15:15,FRIDAY_TEACHING,JOURNAL_CLUB,***TBD (abhängig von Präsenz)***,***Thema folgt***
4,2026-01-30,FR,14:45-15:15,FRIDAY_TEACHING,BEDSIDE_INFEKTIOLOGIE,***TBD (abhängig von Präsenz)***,***Thema folgt***
5,2026-02-06,FR,14:45-15:15,FRIDAY_TEACHING,JOURNAL_CLUB,***TBD (abhängig von Präsenz)***,***Thema folgt***
6,2026-02-13,FR,14:45-15:15,FRIDAY_TEACHING,BEDSIDE_INFEKTIOLOGIE,***TBD (abhängig von Präsenz)***,***Thema folgt***
7,2026-02-20,FR,14:45-15:15,FRIDAY_TEACHING,JOURNAL_CLUB,***TBD (abhängig von Präsenz)***,***Thema folgt***
8,2026-02-27,FR,14:45-15:15,FRIDAY_TEACHING,BEDSIDE_INFEKTIOLOGIE,***TBD (abhängig von Präsenz)***,***Thema folgt***
9,2026-03-06,FR,14:45-15:15,FRIDAY_TEACHING,JOURNAL_CLUB,***TBD (abhängig von Präsenz)***,***Thema folgt***


## Holidays exclusion

In [135]:
!pip install holidays


In [136]:
import holidays

def swiss_holidays(year: int):
    return holidays.Switzerland(years=year)

CH_HOLIDAYS_2026 = swiss_holidays(2026)

In [137]:
def exclude_holidays(df, date_col="date"):
    return df[~df[date_col].isin(CH_HOLIDAYS_2026)]


In [138]:
### apply and print when applied

# Plan

In [149]:
FULL_SCHEDULE = pd.concat(
    [
        kimsim_2026,
        interprof_wb_2026,
        tuesday_morning_2026,
        imc_update_2026,
        teaching_tuesday_2026,
        wednesday_2026,
        thursdays_2026,
        fridays_2026,
    ],
    ignore_index=True,
)

FULL_SCHEDULE = exclude_holidays(FULL_SCHEDULE)


In [150]:
def normalize_schedule(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # Datum
    df["Datum"] = df["date"]

    # Zeit
    df["Zeit"] = df.get("time")

    # Verantwortliche (Priorität)
    df["Verantwortliche"] = (
        df.get("verantwortliche_final")
        .combine_first(df.get("verantwortliche"))
    )

    # Thema (Priorität)
    df["Thema"] = (
        df.get("thema_final")
        .combine_first(df.get("thema"))
    )

    # Ort (optional!)
    if "ort" in df.columns:
        df["Ort"] = df["ort"]
    else:
        df["Ort"] = pd.NA

    return df[["Datum", "Zeit", "Verantwortliche", "Thema", "Ort"]]
FULL_SCHEDULE_CLEAN = normalize_schedule(FULL_SCHEDULE)


In [151]:
FULL_SCHEDULE_CLEAN = exclude_holidays(
    FULL_SCHEDULE_CLEAN,
    date_col="Datum",
)


In [152]:
FULL_SCHEDULE_CLEAN = (
    FULL_SCHEDULE_CLEAN
    .sort_values(["Datum", "Zeit"])
    .reset_index(drop=True)
)


In [154]:
FULL_SCHEDULE_CLEAN.head(60)

,Datum,Zeit,Verantwortliche,Thema,Ort
0,2026-01-02,14:45-15:15,***TBD (abhängig von Präsenz)***,***Thema folgt***,NaN
1,2026-01-05,14:45-15:30,,Interprofessionelle Weiterbildung,NaN
2,2026-01-06,11:30-11:45,***KEIN SENIOR VERFÜGBAR***,None,NaN
3,2026-01-06,17:30-18:30,Nadja Schai,Wie entwickeln sich unsere neuro(chirurgischen...,OPO E 218
4,2026-01-07,14:30-15:15,***TBD (Mittwochskurrikulum)***,***Thema folgt***,NaN
5,2026-01-08,13:00-13:45,Fall-führende Ärzteschaft,Interprofessionelle Therapieplanung,NaN
6,2026-01-09,14:45-15:15,***TBD (abhängig von Präsenz)***,***Thema folgt***,NaN
7,2026-01-12,07:30-11:15,,KimSim ICU oder IMC,NaN
8,2026-01-12,12:30-16:15,,KimSim ICU oder IMC,NaN
9,2026-01-12,14:45-15:30,,Interprofessionelle Weiterbildung,NaN
